In [2]:
import pandas as pd

df = pd.read_csv("Datasets/retail_data.csv")

df.shape

(302010, 30)

In [13]:
df.head()

,Transaction_ID,Customer_ID,Name,Email,Phone,Address,City,State,Zipcode,Country,...,Total_Amount,Product_Category,Product_Brand,Product_Type,Feedback,Shipping_Method,Payment_Method,Order_Status,Ratings,products
0,8691788.0,37249.0,Michelle Harrington,Ebony39@gmail.com,1.414787e+09,3959 Amanda Burgs,Dortmund,Berlin,77985.0,Germany,...,324.086270,Clothing,Nike,Shorts,Excellent,Same-Day,Debit Card,Shipped,5.0,Cycling shorts
1,2174773.0,69749.0,Kelsey Hill,Mark36@gmail.com,6.852900e+09,82072 Dawn Centers,Nottingham,England,99071.0,UK,...,806.707815,Electronics,Samsung,Tablet,Excellent,Standard,Credit Card,Processing,4.0,Lenovo Tab
2,6679610.0,30192.0,Scott Jensen,Shane85@gmail.com,8.362160e+09,4133 Young Canyon,Geelong,New South Wales,75929.0,Australia,...,1063.432799,Books,Penguin Books,Children's,Average,Same-Day,Credit Card,Processing,2.0,Sports equipment
3,7232460.0,62101.0,Joseph Miller,Mary34@gmail.com,2.776752e+09,8148 Thomas Creek Suite 100,Edmonton,Ontario,88420.0,Canada,...,2466.854021,Home Decor,Home Depot,Tools,Excellent,Standard,PayPal,Processing,4.0,Utility knife
4,4983775.0,27901.0,Debra Coleman,Charles30@gmail.com,9.098268e+09,5813 Lori Ports Suite 269,Bristol,England,48704.0,UK,...,248.553049,Grocery,Nestle,Chocolate,Bad,Standard,Cash,Shipped,1.0,Chocolate cookies


In [14]:
df.columns

Index(['Transaction_ID', 'Customer_ID', 'Name', 'Email', 'Phone', 'Address',
       'City', 'State', 'Zipcode', 'Country', 'Age', 'Gender', 'Income',
       'Customer_Segment', 'Date', 'Year', 'Month', 'Time', 'Total_Purchases',
       'Amount', 'Total_Amount', 'Product_Category', 'Product_Brand',
       'Product_Type', 'Feedback', 'Shipping_Method', 'Payment_Method',
       'Order_Status', 'Ratings', 'products'],
      dtype='object')

In [21]:
df['products'].nunique()

318

In [22]:
df['Product_Category'].nunique()

5

In [23]:
df['Product_Category'].unique()

array(['Clothing', 'Electronics', 'Books', 'Home Decor', 'Grocery', nan],
      dtype=object)

In [26]:
null_rows_count = df.isnull().any(axis=1).sum()

print(null_rows_count)

8099


In [28]:
df.shape

(302010, 30)

In [83]:
customer_info = df[['Customer_ID', 'Name', 'Email', 'Phone', 'Address', 'City', 'State', 'Zipcode', 'Country', 'Age', 'Gender', 'Income', 'Customer_Segment']]
transaction_details = df[['Transaction_ID', 'Customer_ID', 'Date', 'Year', 'Month', 'Time', 'Total_Purchases', 'Amount', 'Total_Amount']]
transaction_logistics = df[['Transaction_ID', 'Shipping_Method', 'Payment_Method', 'Order_Status']]

# Generate a unique Product_ID for each unique product with leading zeros
unique_products = df['products'].unique()
product_mapping = {product: f'P{idx:03}' for idx, product in enumerate(unique_products)}

# Map the Product_IDs back to the dataframe
df['Product_ID'] = df['products'].map(product_mapping)

# Create product_info with Product_ID and without Transaction_ID
product_info = df[['Product_ID', 'Product_Category', 'Product_Brand', 'Product_Type', 'products']]

# Add Product_ID to feedback
feedback = df[['Product_ID', 'Feedback', 'Ratings']]

# Verify the data
customer_info.head()

,Customer_ID,Name,Email,Phone,Address,City,State,Zipcode,Country,Age,Gender,Income,Customer_Segment
0,37249.0,Michelle Harrington,Ebony39@gmail.com,1.414787e+09,3959 Amanda Burgs,Dortmund,Berlin,77985.0,Germany,21.0,Male,Low,Regular
1,69749.0,Kelsey Hill,Mark36@gmail.com,6.852900e+09,82072 Dawn Centers,Nottingham,England,99071.0,UK,19.0,Female,Low,Premium
2,30192.0,Scott Jensen,Shane85@gmail.com,8.362160e+09,4133 Young Canyon,Geelong,New South Wales,75929.0,Australia,48.0,Male,Low,Regular
3,62101.0,Joseph Miller,Mary34@gmail.com,2.776752e+09,8148 Thomas Creek Suite 100,Edmonton,Ontario,88420.0,Canada,56.0,Male,High,Premium
4,27901.0,Debra Coleman,Charles30@gmail.com,9.098268e+09,5813 Lori Ports Suite 269,Bristol,England,48704.0,UK,22.0,Male,Low,Premium


In [84]:
# Verify the data
transaction_details.head()

,Transaction_ID,Customer_ID,Date,Year,Month,Time,Total_Purchases,Amount,Total_Amount
0,8691788.0,37249.0,9/18/2023,2023.0,September,22:03:55,3.0,108.028757,324.086270
1,2174773.0,69749.0,12/31/2023,2023.0,December,8:42:04,2.0,403.353907,806.707815
2,6679610.0,30192.0,4/26/2023,2023.0,April,4:06:29,3.0,354.477600,1063.432799
3,7232460.0,62101.0,05-08-23,2023.0,May,14:55:17,7.0,352.407717,2466.854021
4,4983775.0,27901.0,01-10-24,2024.0,January,16:54:07,2.0,124.276524,248.553049


In [85]:
# Verify the data
transaction_logistics.head()

,Transaction_ID,Shipping_Method,Payment_Method,Order_Status
0,8691788.0,Same-Day,Debit Card,Shipped
1,2174773.0,Standard,Credit Card,Processing
2,6679610.0,Same-Day,Credit Card,Processing
3,7232460.0,Standard,PayPal,Processing
4,4983775.0,Standard,Cash,Shipped


In [86]:
# Verify the data
product_info.head()

,Product_ID,Product_Category,Product_Brand,Product_Type,products
0,P000,Clothing,Nike,Shorts,Cycling shorts
1,P001,Electronics,Samsung,Tablet,Lenovo Tab
2,P002,Books,Penguin Books,Children's,Sports equipment
3,P003,Home Decor,Home Depot,Tools,Utility knife
4,P004,Grocery,Nestle,Chocolate,Chocolate cookies


In [87]:
# Verify data
feedback.head()

,Product_ID,Feedback,Ratings
0,P000,Excellent,5.0
1,P001,Excellent,4.0
2,P002,Average,2.0
3,P003,Excellent,4.0
4,P004,Bad,1.0


In [88]:
# Display the range of Product_IDs
product_ids = df['Product_ID'].unique()
print("Range of Product_IDs:", min(product_ids), "to", max(product_ids))

# Count of unique Product_IDs
print("Total unique Product_IDs:", len(product_ids))

# Verify the number of unique products
print("Total unique products:", len(unique_products))

Range of Product_IDs: P000 to P317
Total unique Product_IDs: 318
Total unique products: 318


In [64]:
# Save to CSV files
customer_info.to_csv('customer_info.csv', index=False)
transaction_details.to_csv('transaction_details.csv', index=False)
product_info.to_csv('product_info.csv', index=False)
feedback.to_csv('feedback.csv', index=False)
transaction_logistics.to_csv('transaction_logistics.csv', index=False)

   Customer_ID                 Name                Email         Phone  \
0      37249.0  Michelle Harrington    Ebony39@gmail.com  1.414787e+09   
1      69749.0          Kelsey Hill     Mark36@gmail.com  6.852900e+09   
2      30192.0         Scott Jensen    Shane85@gmail.com  8.362160e+09   
3      62101.0        Joseph Miller     Mary34@gmail.com  2.776752e+09   
4      27901.0        Debra Coleman  Charles30@gmail.com  9.098268e+09   

                       Address        City            State  Zipcode  \
0            3959 Amanda Burgs    Dortmund           Berlin  77985.0   
1           82072 Dawn Centers  Nottingham          England  99071.0   
2            4133 Young Canyon     Geelong  New South Wales  75929.0   
3  8148 Thomas Creek Suite 100    Edmonton          Ontario  88420.0   
4    5813 Lori Ports Suite 269     Bristol          England  48704.0   

     Country   Age  Gender Income Customer_Segment  
0    Germany  21.0    Male    Low          Regular  
1         UK  19